# Water Sort Solver

This code is designed to:
 - Read a screenshot of the Water Sort puzzle game's starting state
 - Output a list of steps to solve the given puzzle
 
 This code does not handle the "mystery" puzzles where only the topmost color of each tube is shown.

In [ ]:
# SETUP
from PIL import Image, ImageFilter
HEIGHT = 4

## Function: similarColor(color,compare)
This is a quick little utility function intended to make the script somewhat resilient to minor variations in color of the screenshot.
**Input** - Two tuples containing three integers each.

In [ ]:
def similarColor(color,compare):
    assert type(color)==tuple, "color must be a tuple; given: %s"%str(color)
    assert type(compare)==tuple, "compare must be a tuple; given: %s"%str(compare)
    assert (type(color[0])==int and type(color[1])==int and type(color[2])==int 
            and color[0]>=0 and color[1]>=0 and color[2]>=0
            and color[1]<=255 and color[1]<=255 and color[2]<=255), 
        "color must be a valid RGB tuple with three integers between 0 and 255; given: %s"%str(color)
    if (abs(color[0]-compare[0])<=10
        and abs(color[1]-compare[1])<=10
        and abs(color[1]-compare[1])<=10): return True
    else: return False

In [ ]:

def pix(data,w,i,j):
    if i*w+j>len(data): return False
    return data[i*w+j]
def whiteCountRow(data,w,row):
    row=[pix(data,w,row,x) for x in range(w)]
    ct=0        
    for pixel in row: 
        if pixel>250: ct+=1
    return ct    
def whiteCountCol(data,w,h,col):
    col=[pix(data,w,x,col) for x in range(h)]
    ct=0        
    for pixel in col: 
        if pixel>250: ct+=1
    return ct    
def colored(r, g, b, text):
    return "\033[38;2;{};{};{}m{}\033[38;2;255;255;255m".format(r, g, b, text)
def findTubes(imagePath):
    assert imagePath[-4:]==".png", "findTubes requires a .png file. (%s) provided."%imagePath[-4:]
    pathRoot=imagePath[:-4]
    #print("path root: %s"%pathRoot)
    miniPath=pathRoot+"_mini.png"
    edgePath=pathRoot+"_edge.png"
    pieceRoot=pathRoot+"piece_"
    allTubesPath=pathRoot+"_all.png"
    allTubesEdgesPath=pathRoot+"_all_edges.png"

    img = Image.open(imagePath)
    img = img.convert("RGB")
    
    scale=8
    img=img.resize((int(img.width/scale),int(img.height/scale)),0)
    #img=img.crop((2,2,img.width-2,img.height-2))
    img = img.quantize(40)
    img = img.convert("RGB")
    d = img.getdata()
    #Clean background
    flat=[item for item in d]
    for i in range(len(flat)):
        if flat[i][0]<90 and flat[i][1]<90 and flat[i][2]<90: flat[i]=(0,0,0)
    img.putdata(flat)
    
    img.save(miniPath)
    w = img.width
    h = img.height

    #img=
    w=img.width
    h=img.height
    data=img.getdata()
    edge = img.convert("L") # convert to grayscale
    edge = edge.filter(ImageFilter.FIND_EDGES) # find edges
    edge.save(edgePath) # for reference
    edata=edge.getdata()

        
    pieces=[[]]
    for row in range(h):
        wc=whiteCountRow(edata,w,row)
        #print("row: %d, wc: %d"%(row,wc))
        if wc<1: 
            last=0
            continue
        else: 
            if last==0: pieces.append([])
            last=1
            pieces[-1]+=[data[i] for i in range(w*row,w*row+w)]
    pieces.remove([])
    #print(len(pieces))
    #for piece in pieces:
        #print(len(piece))
    for i in range(len(pieces)):
        dh=len(pieces[i])//w
        #print("w: %d' h: %d; data: XXX"%(w,dh))
        newimg=Image.new("RGB",(w,dh))
        newimg.putdata(pieces[i])
        newimg.save("%s%d.png"%(pieceRoot,i))        

    assert len(pieces[2])==len(pieces[3]), "Sections 2 and 3 should be the same height"
    #tubes are pieces 2 and 3
    newh=len(pieces[2])//w
    allTubes=Image.new("RGB",(2*w+5,newh))
    newData=[]
    for row in range(newh):
        newData+=[pieces[2][row*w+x] for x in range(w)]
        newData+=[(0,0,0) for i in range(5)]
        newData+=[pieces[3][row*w+x] for x in range(w)]
    allTubes.putdata(newData)
    allTubes.save(allTubesPath)
    
    allTubesEdges=allTubes.convert("L")
    allTubesEdges = allTubesEdges.filter(ImageFilter.FIND_EDGES) # find edges
    allTubesEdges.save(allTubesEdgesPath) # for reference
    
    
    atw=allTubes.width
    ath=allTubes.height
    atd=allTubes.getdata()
    testout=allTubes.convert("RGB")
    ate=allTubesEdges.getdata()
    ate=[255 if item>200 else 0 for item in ate]
    
    
    tubeCols=[]
    last=0
    for col in range(atw):
        wc=whiteCountCol(ate,atw,ath,col)
        #print("col: %d, wc: %d"%(col,wc))
        if wc<1: 
            last=0
            continue
        else: 
            if last==0: tubeCols.append([col,col])
            last=1
            tubeCols[-1][1]=col
    #print(tubeCols)
    
    
    
    #for tube in tubeCols: print(tube,tube[1]-tube[0])
    tubeWidths=[tube[1]-tube[0] for tube in tubeCols]
    #print(sum(tubeWidths)/len(tubeWidths))
    tubeImgs=[]
    for i in range(len(tubeCols)):
        tubeData=[]
        for j in range(ath):
            tubeData+=[atd[atw*j+k] for k in range(tubeCols[i][0],tubeCols[i][1]+1)]
        tubeImg=Image.new("RGB",(tubeCols[i][1]+1-tubeCols[i][0],ath))
        tubeImg.putdata(tubeData)
        tubeImg.save("%stube%d.png"%(pathRoot,i))
        tubeImgs.append(tubeImg)
        
    tubes=[]
    for tubeImg in tubeImgs:
        td=tubeImg.getdata()
        tw=tubeImg.width
        th=tubeImg.height
        ctr=tw//2
        contents=[]
        for row in range(th):
            col=pix(td,tw,row,ctr)
            if len(contents)==0: contents.append({"color":col,"height":1})
            else:
                if contents[-1]["color"]==col: contents[-1]["height"]+=1
                else: contents.append({"color":col,"height":1})
        tubes.append(contents)
    final=[]
    for tube in tubes:
        rem=[]
        fin=[]
        height=sum([item["height"] for item in tube])
        for item in tube:
            if item["height"]<height/10 or item["color"]==(0,0,0) : rem.append(item)
        for item in rem: tube.remove(item)
        height=sum([item["height"] for item in tube])
        for i in range(len(tube)):
            if abs(height/4-tube[i]["height"])<=3: 
                fin.append(tube[i]["color"])
            elif abs(height/2-tube[i]["height"])<=3: 
                fin.extend([tube[i]["color"],tube[i]["color"]])
            elif abs(3*height/4-tube[i]["height"])<=3: 
                fin.extend([tube[i]["color"],tube[i]["color"],tube[i]["color"]])
        fin.reverse()
        final.append(fin)
    colorMap={}
    unkn=0
    for tube in final:
        for color in tube:
            if color not in colorMap: 
                colorMap[color]={}
                if similarColor(color,(217, 103, 124)):
                    colorMap[color]["name"]="Pink"
                    colorMap[color]["abbr"]="Pi"
                elif similarColor(color,(99, 100, 101)):
                    colorMap[color]["name"]="Gray"
                    colorMap[color]["abbr"]="Gy"
                elif similarColor(color,(131, 211, 134)):
                    colorMap[color]["name"]="Lime"
                    colorMap[color]["abbr"]="Li"
                elif similarColor(color,(105, 161, 223)):
                    colorMap[color]["name"]="Cyan"
                    colorMap[color]["abbr"]="Cy"
                elif similarColor(color,(134, 153, 68)):
                    colorMap[color]["name"]="Olive"
                    colorMap[color]["abbr"]="Ol"
                elif similarColor(color,(105, 48, 142)):
                    colorMap[color]["name"]="Purple"
                    colorMap[color]["abbr"]="Pu"
                elif similarColor(color,(175, 58, 43)):
                    colorMap[color]["name"]="Red"
                    colorMap[color]["abbr"]="Rd"
                elif similarColor(color,(58, 49, 174)):
                    colorMap[color]["name"]="Blue"
                    colorMap[color]["abbr"]="Bl"
                elif similarColor(color,(219, 144, 81)):
                    colorMap[color]["name"]="Orange"
                    colorMap[color]["abbr"]="Or"
                elif similarColor(color,(119, 77, 26)):
                    colorMap[color]["name"]="Brown"
                    colorMap[color]["abbr"]="Br"
                elif similarColor(color,(237, 219, 110)):
                    colorMap[color]["name"]="Yellow"
                    colorMap[color]["abbr"]="Ye"
                elif similarColor(color,(46, 99, 56)):
                    colorMap[color]["name"]="Green"
                    colorMap[color]["abbr"]="Gn"
                else:
                    unkn+=1
                    colorMap[color]["name"]="Unknown %s"%str(chr(64+unkn))
                    colorMap[color]["abbr"]="X%s"%str(chr(64+unkn))
    return {"colorMap":colorMap,"tubes":final}
        

def solveFromScreenshot(img):
    results=findTubes(img)
    colorMap=results["colorMap"]
    parsedTubes=results["tubes"]

    char=65
    charToRgb={}
    for color in colorMap:
        colorMap[color]["char"]=chr(char)
        charToRgb[chr(char)]={"rgb":color,"abbr":colorMap[color]["abbr"],"name":colorMap[color]["name"]}
        char+=1

    tubes=[[colorMap[col]["char"] for col in tube] for tube in parsedTubes]
    res=solve(tubes)
    if res["solved"]==True:
        path=res["steps"]
        for item in path: showFprintColorStacked(charToRgb,item)
    else:
        paths=res["paths"]
        print(paths)

def showFprintColors(charToRgb,fprint):
    out=""
    for char in fprint:
        if char in charToRgb: out+=colored(charToRgb[char]["rgb"][0],charToRgb[char]["rgb"][1],charToRgb[char]["rgb"][2],char)
        else: out+=char
    print(out)

def showFprintColorStacked(charToRgb,fprint):
    out=""
    tubes=[[fprint[i*(HEIGHT+1)+j] for j in range(HEIGHT)] 
           for i in range((len(fprint)+1)//(HEIGHT+1))]
    for i in range(HEIGHT):
        index=HEIGHT-1-i
        for j in range(len(tubes)):
            char=tubes[j][index]
            if char in charToRgb:
                out+=colored(charToRgb[char]["rgb"][0],charToRgb[char]["rgb"][1],charToRgb[char]["rgb"][2],char)+" "
            else: out+=char+" "
        out+="\n"
    print(out)

## Function: fprintFromTubes(tubes)
This function is used to generate a "fingerprint" from a list of tubes.
**Input** - A two-dimensional array (list of lists) where each list contains single-character strings which represent the colors in a test tube from bottom to top.
**Output** - A string representing the tubes, in order, separated by dashes. Empty sections of tube are represented as '0'.

## Function: fprintFromTube(tube)
This is a helper function called from `fprintFromTubes(tubes)` to generate the fingerprint for each individual tube.
**Input** - A list of single-character strings item represents a colors in a test tube from bottom to top.
**Output** - A string representing the tube, in order.

In [ ]:
def fprintFromTubes(tubes):
    assert type(tubes)==list, "tubes must be a list - %s"%tubes
    assert type(tubes[0])==list, "tubes must be a list of tubes - %s" %tubes[0]
    for i in range(len(tubes)):
        if i==0: fp=fprintFromTube(tubes[i])
        else: fp+="-"+fprintFromTube(tubes[i])
    flen=HEIGHT*len(tubes)+len(tubes)-1    
    if len(fp)!=flen: return False
    return fp
def fprintFromTube(tube):
    assert type(tube)==list, "tube must be a list of colors - %s" %tube
    fp="".join(tube)
    while len(fp)<HEIGHT: fp+="0"
    if len(fp)!=HEIGHT: return False
    return fp

## Function: equivFromTubes(tubes) 
This function is used to return a standardized "fingerprint" from a list of tubes. This will output the same fingerprint for any combination of tubes that is equivalent. E.g. if on the first move, you pour from the first tube into either of the empty tubes, the result is the same, even though the absolute position differs. This function is used to check that we're not doing repeated effort by continuing to process duplicate states.
**Input** - A two-dimensional array (list of lists) where each list represents the colors in a test tube from bottom to top.
**Output** - A string representing the tubes, in **alphabetical** order, separated by dashes. Empty sections of tube are represented as '0'.

In [ ]:
def equivFromTubes(tubes):
    assert type(tubes)==list, "tubes must be a list - %s"%tubes
    assert type(tubes[0])==list, "tubes must be a list of tubes - %s" %tubes[0]
    tubes.sort()
    return fprintFromTubes(tubes)

## Function: solvedFromFprint(fprint)
This function returns the standardized "final" solved state of the puzzle. 
**Input** - A string representing the tubes, in order, separated by dashes. Empty sections of tube are represented as '0'.
**Output** - A string representing the tubes, when fully solved, in alphabetical order, separated by dashes. Empty sections of tube are represented as '0'.

In [ ]:
def solvedFromFprint(fprint):
    assert type(fprint)==str, "solvedFromFprint requires a fingerprint string, not %s" %fprint
    chars=[char for char in fprint]
    while "-" in chars: chars.remove("-")
    chars.sort()
    for i in range((len(fprint)+1)//(HEIGHT+1)):
        if i==0: fp="".join(chars[0:HEIGHT])
        else: fp+="-"+"".join(chars[i*(HEIGHT):i*(HEIGHT)+HEIGHT])
    return fp

## Function: tubesFromFprint(fprint)
This method converts a full fingerprint back into a list of lists.
**Input** - A string representing the tubes, in order, separated by dashes. Empty sections of tube are represented as '0'.
**Output** - A two-dimensional array (list of lists) where each list contains single-character strings which represent the colors in a test tube from bottom to top.

## Function: tubeFromFprint(fprint)
This is a helper function called from `tubesFromFprint(fprint)` to convert a fingerprint to a list for each individual tube.
**Input** - A string representing the tube, in order.
**Output** - A list of single-character strings item represents a colors in a test tube from bottom to top.

In [ ]:
def tubesFromFprint(fprint):
    assert type(fprint)==str, "fprint must be a string - %s" %fprint
    tubes=[]
    for i in range((len(fprint)+1)//(HEIGHT+1)):
        tubes.append(tubeFromFprint(fprint[i*(HEIGHT+1):HEIGHT+i*(HEIGHT+1)]))
    return tubes
def tubeFromFprint(fprint):
    assert type(fprint)==str, "fprint must be a string - %s" %fprint
    tube=[]
    for char in fprint:
        if char!="0": tube.append(char)
    return tube 

## Function: equivFromFprint(fprint)
This function returns the standardized fingerprint for a state from a non-standardized fingerprint.
**Input** - A string representing the tubes, in order, separated by dashes. Empty sections of tube are represented as '0'.
**Output** - A string representing the tubes, in **alphabetical** order, separated by dashes. Empty sections of tube are represented as '0'.

In [ ]:
def equivFromFprint(fprint):
    assert type(fprint)==str, "fprint must be a string - %s" %fprint
    return equivFromTubes(tubesFromFprint(fprint))

## Function: pour(fprint,fr,to)
This function is one of the major workhorses of the solution side of this script. It takes the fingerprint of a state and the index of two tubes: the one to pour **fr**om and the one to pour **to**. 

If the operation can't be performed, this method returns `False`. This happens if:
- The **fr** and **to** values are the same.
- The **fr** tube is empty.
- The **to** tube is full.
- The **to** tube isn't empty and the top color of the **fr** tube doesn't match the top color of the **to** tube.

If the operation can be performed, this method returns the fingerprint of the resulting state: what does the puzzle look like after you've poured from **fr** to **to**?

**Input** - 
- A fingerprint string
- An integer < the number of tubes
- Another integer < the number of tubes
**Output** - 
- `False` if the operation is invalid.
- A fingerprint string if the operation is valid.

In [ ]:
def pour(fprint,fr,to):
    assert type(fprint)==str, "fprint must be a string - %s" %fprint
    if fr==to: return False # Null operation
    tubes=tubesFromFprint(fprint)
    if len(tubes[fr])==0: 
        # Can't pour from an empty tube   
        return False
    if len(tubes[to])==HEIGHT: 
        # Can't pour into a full tube
        return False
    color=tubes[fr][-1]
    if len(tubes[to])>0 and tubes[to][-1]!=color: 
        # Can't pour with mismatched colors
        return False
    qty=0 # How many blocks of continuous color can we pour?
    for i in range(1,1+len(tubes[fr])):
        if tubes[fr][-i]==tubes[fr][-1]: qty+=1
        else: break # This segment is a different color
    while qty>0 and len(tubes[to])<HEIGHT:
        tubes[to].append(tubes[fr].pop())
        qty-=1
    return fprintFromTubes(tubes)

## Function: solve(tubes)
This function is the other major workhorse of the solution side of the script. This method takes a starting game state as a two-dimensional array and solves it by brute-force methods.

To do so, this method:
- Tracks all possible states it has discovered as a list of fingerprints `toExplore`
- Tracks all `paths` that the game can take from state to state. This is a dictionary with fingerprints as its keys. Each value is another dictionary with keys `"from"` and `"to"`, to track states that link to this one (incoming or outgoing, respectively).
- For each possible state in `toExplore`, attempt all possible `pour` actions from each tube to each other tube
- At each step, check to see whether:
  - The puzzle has been solved - if so, note the final solution fingerprint.
  - The resulting state has already been explored - if so, skip to the next.
This traverses all possible game states, tracking how to get from one to the next. It also prioritizes finding the **shortest path** to each state, so inefficient paths are effectively ignored. 

**Input** - A two-dimensional array representing the starting game state.
**Output** - 
- If the puzzle could not be solved: a dictionary with `"solved":False` and `"paths"` containing the dictionary of all states and the connections between them.
- If the puzzle could be solved: a dictionary with `"solved":True` and `"steps"` containing a list representing the shortest path from the initial state to solved, step by step, as fingerprints.

In [ ]:
def solve(tubes):
    assert type(tubes)==list, "tubes must be a list - %s"%tubes
    assert type(tubes[0])==list, "tubes must be a list of tubes - %s" %tubes[0]
    fprint=fprintFromTubes(tubes)
    assert type(fprint)==str, "fprint must be a str - %s"%fprint
    solved=solvedFromFprint(fprint)
    toExplore=[fprint]
    explored=[]
    isSolved=False
    paths={fprint:{"from":False,"to":[]}}
    while isSolved==False and len(toExplore)>0:
        thisfp=toExplore.pop()
        explored.append(equivFromFprint(thisfp))
        theseTubes=tubesFromFprint(thisfp)
        for i in range(len(theseTubes)):
            for j in range(len(theseTubes)):
                if i==j: continue
                thispour=pour(thisfp,i,j)
                if thispour==False: continue
                equiv=equivFromFprint(thispour)
                if equiv in explored: continue
                toExplore.append(thispour)
                paths[thisfp]["to"].append(thispour)
                if thispour not in paths:
                    paths[thispour]={"from":[thisfp],"to":[]}
                else: 
                    paths[thispour]["from"].append(thisfp)
                if equiv==solved: 
                    isSolved=True
                    solved=thispour
    if isSolved==False:
        print("Somehow we ended the loop without solving the puzzle.")
        return {"solved":False,"paths":paths}
    steps=[solved]
    while paths[steps[-1]]["from"]!=False:
        steps.append(paths[steps[-1]]["from"][0])
    steps.reverse()
    return {"solved":True,"steps":steps}

In [ ]:






        




for item in ["lvl990.png","lvl1003.png","lvl1002.png",
             "lvl1006.png","lvl1008.png","lvl1009.png","lvl1011.png"]:
    print()
    print(item)
    solveFromScreenshot(item)



